In [55]:
import os
import cv2
import shutil
from glob import glob

In [56]:
dir_labels = "/home/masum/Downloads/widerface/wider_face_split/wider_face_train_bbx_gt.txt"
dir_images = "/home/masum/Downloads/widerface/WIDER_train/images/"
dir_out = "/home/masum/Downloads/widerface/WIDER_train/out/"

In [57]:
def getLabel(filename):
    faces = {}
    bbox = []
    with open(filename) as lines:
        for line in lines:
            line = line.strip()
            if line.find(".jpg") >= 0:
                if len(bbox) > 0:
                    faces[name] = bbox
                    bbox = []
                basename = os.path.basename(line)
                name = basename[:-4]
                lines.readline() #read number of bbox
            else:
                line = line.split() # extract bbox info
                x = line[0]
                y = line[1]
                w = line[2]
                h = line[3]
                bbox.append([x,y,w,h])
        faces[name] = bbox #last bbox
        return faces        

In [58]:
def setLabel(imgH, imgW, filename, bboxes):
    f = open(filename + ".txt", "w")
    for bbox in bboxes:
        x = float(bbox[0])
        y = float(bbox[1])
        w = float(bbox[2])
        h = float(bbox[3])
                    
        if x <= 0.0: 
            x = 0.001
        if y <= 0.0:
            y = 0.001
        if w <= 0.0:
            w = 0.001
        if h <= 0.0:
            h = 0.001
            
        center_x = x + 0.5 * w
        center_y = y + 0.5 * h
    
        X = center_x / imgW
        Y = center_y / imgH
        W = w / imgW
        H = h / imgH
        f.write("{} {:.2f} {:.2f} {:.2f} {:.2f}\n".format(0 ,X, Y, W, H))
    f.close()

In [59]:
labels = getLabel(dir_labels)

In [60]:
folders = os.listdir(dir_images)
for folder in folders:
    temp = os.path.join(dir_images,folder)
    images = glob(os.path.join(temp,"*.jpg"))
    
    for img in images:
        basename = os.path.basename(img)
        name = basename[:-4]
        
        im = cv2.imread(img)
        imgH, imgW, imgc = im.shape
        
        setLabel(imgH, imgW, dir_out+name, labels[name])
        shutil.copy(img, dir_out)
